In [40]:
# device = 'K0303'
# active_power_variable = 'active_power'
path_base_r_string = r'D:\OneDrive - CELSIA S.A E.S.P'

%load_ext autoreload
%autoreload 2
import warnings
# warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
# this cell enables project path relative imports
import sys
from pathlib import Path
path_base = Path(path_base_r_string)
project_path = path_base / 'Proyectos' / 'Eficiencia_Energetica' / 'Bancolombia' / 'Experimental'
sys.path.append(str(project_path))

In [42]:
# import all your modules here
import json
import pandas as pd
import numpy as np
import config as cfg
import library_report as Report

In [43]:
BLACKLISTED_VARIABLE_LABELS = (
    "ilu-consumo-activa",
    "consumo-domingo",
    "consumo-sabado",
    "consumo-semana",
    "potencia-diaria-promedio",
    "tipo-dia",
    "hora-def",
    "consumo-energia-reactiva-total",
    "aa-consumo-activa",
    "front-consumo-activa",
    # "c2-energia-reactiva-capacitiva-acumulada",
    # "c2-factor-de-potencia",
    # "c2-corriente-total",
    # "c2-corriente-l3",
    # "c2-corriente-l2",
    # "c2-corriente-l1",
    # "c2-tension-fase---neutro-l3",
    # "c2-tension-fase---neutro-l2",
    # "c2-tension-fase---neutro-l1",
    # "c2-frecuencia",
    # "c2-potencia-reactiva-instantanea-total",
    "aa-potencia-activa",
    # "c2-energia-reactiva-inductiva-acumulada",
    # "c2-energia-activa-acumulada",
    # "c1-factor-de-potencia",
    # "c1-corriente-total",
    # "c1-corriente-l3",
    # "c1-corriente-l2",
    # "c1-corriente-l1",
    "front-tension-3",
    "front-tension-2",
    "front-tension-1",
    # "c1-frecuencia",
    # "c1-potencia-reactiva-instantanea-total",
    "front-potencia-activa",
    # "c1-energia-reactiva-capacitiva-acumulada",
    # "c1-energia-reactiva-inductiva-acumulada",
    # "c1-energia-activa-acumulada",
    # "c1-tension-fase---neutro-l3",
    # "c1-tension-fase---neutro-l2",
    # "c1-tension-fase---neutro-l1",
    "otros-consumo-activa",
    "ilu-potencia-activa",
    "consumo-energia-reactiva-total-1",
    "consumo-sabados",
    "energia-reactiva-inductiva",
    "energia-activa",
    "factor-de-potencia",
    "corriente-total",
    "corriente-l3",
    "corriente-l2",
    "corriente-l1",
    "frecuencia",
    "potencia-reactiva-instantanea-total",
    "energia-reactiva-capacitiva-acumulada",
    "energia-reactiva-inductiva-acumulada",
    "energia-activa-acumulada",
    # "c2-potencia-activa-instantanea-total",
    # "c1-potencia-activa-instantanea-total",
    "consumo-promedio-diario",
    "potencia-diario-promedio",
    # "c2-potencia-activa",
    "potencia-promedio-diaria",
    "potencia-promedio-diario",
    "consumo-promedio",
    "consumo-energia-reactiva-total-2",
    # "c2-consumo-activa",
    "ilu-consumo-activa-1",
    "new-variable",
    "consumo-front-sintetica",
    "consumo-activa-total",
    "new-variable-7",
    "er-inductiva-acumulada",
    "new-variable-29",
    "new-variable-28",
    "new-variable-23",
    "new-variable-22",
    "new-variable-21",
    "new-variable-20",
    "new-variable-19",
    "new-variable-18",
    "new-variable-16",
    "new-variable-15",
    "new-variable-14",
    "new-variable-13",
    "new-variable-12",
    "new-variable-11",
    "new-variable-10",
    "new-variable-6",
    "front-tension-l2n",
    "front-tension-l1n",
    "new-variable-3",
    "er-capacitiva-acumulada"
)

In [44]:
df_devices = Report.get_available_devices("bancolombia")
df_vars = Report.get_available_variables(list(df_devices['id']))

blacklisted = df_vars['variable_label'].isin(cfg.BLACKLISTED_VARIABLE_LABELS)
VAR_IDS_TO_REQUEST = df_vars.loc[~blacklisted, 'variable_id']
VAR_ID_TO_LABEL = dict(zip(df_vars['variable_id'], df_vars['variable_label']))

In [45]:
CHUNK_SIZE = 100
DATE_INTERVAL_REQUEST = {'start': '2022-09-15', 'end': '2022-10-05'}

df = None
lst_responses = []
for idx in range(0, Report.ceildiv(len(VAR_IDS_TO_REQUEST), CHUNK_SIZE)):
    idx_start = idx * CHUNK_SIZE
    idx_end = (idx + 1) * CHUNK_SIZE
    id_request_subset = VAR_IDS_TO_REQUEST[idx_start:idx_end]

    response = Report.make_request(
        id_request_subset, 
        DATE_INTERVAL_REQUEST, 
    )

    lst_responses.append(response)

df = Report.parse_response(lst_responses, VAR_ID_TO_LABEL)
pd.to_pickle(df, "data_raw.pkl")

In [46]:
df['device_id'] = df['device'].map(dict(zip(df_devices['label'], df_devices['id'])))
df = df.rename(columns={'variable':'variable_label', 'device':'device_label'})

In [68]:
df_latest = (
    df
    .reset_index()
    .groupby(['device_name','device_label','device_id','variable_label'])['datetime']
    .agg('max')
    .reset_index()
)

df_latest = pd.merge(
    df_latest,
    df_vars,
    how='outer'
)

df_latest['device_name'] = df_latest['device_id'].map(dict(zip(df_devices['id'], df_devices['device_name'])))
df_latest['device_label'] = df_latest['device_id'].map(dict(zip(df_devices['id'], df_devices['label'])))
df_latest['delta_days'] = (pd.Timestamp.now().tz_localize('America/Bogota') - df_latest['datetime'])  / np.timedelta64(1, 'D')
df_latest['online']= (df_latest['delta_days'] < 1)

In [94]:
df_latest_wide = df_latest.pivot(index='variable_label', columns='device_label', values='online')
df_report = df_latest_wide.any().to_frame()
df_report = df_report.rename(columns={0:'data_within_past_24h'})
df_report = df_report.reset_index()
df_report['device_name'] = df_report['device_label'].map(dict(zip(df_devices['label'], df_devices['device_name'])))
df_report = df_report.set_index('device_name')

In [95]:
df_report

,device_label,data_within_past_24h
device_name,,
BC 799 - Floridablanca,bc---floridablanca,True
BC 291 - Las Palmas,bc-291-las-palmas,True
BC 90 - Megamall,bc-megamall,True
BC 749 - Santa Monica,bc-santa-monica,False
BC 659 - Girardot,bc01-iluminacion-girardot,True
BC 210 - Banca Colombia Cartagena,bc10,True
BC 678 - Paseo de la Castellana,bc11,False
BC 88 - Cúcuta,bc12,False
BC 185 - Llano Grande Palmira,bc14,True


In [96]:
df_report.to_clipboard()

In [100]:
df_latest_timestamp_report = (
    df_latest
    .groupby(['device_name','device_label','device_id'])['datetime']
    .max()
    .reset_index()
)

df_latest_timestamp_report.to_clipboard()

In [101]:
df_latest_timestamp_report

,device_name,device_label,device_id,datetime
0,BC 044 - Piedecuesta,bc35,622a7291fbcb750090b91137,2022-10-04 13:27:00-05:00
1,BC 061 - Carrera Primera,bc56,622a72f0fbcb750090b91398,2022-10-04 13:15:00-05:00
2,BC 185 - Llano Grande Palmira,bc14,622a724ffbcb750090b90ed6,2022-10-04 13:15:00-05:00
3,BC 197 - Guatapuri,bc6,622a7231fbcb750090b90dee,2022-09-28 10:45:00-05:00
4,BC 205 - Villa Colombia,bc54,622a72e8fbcb750090b9135e,2022-10-04 13:00:00-05:00
5,BC 210 - Banca Colombia Cartagena,bc10,622a723dfbcb750090b90e62,2022-10-04 13:15:00-05:00
6,BC 253 - Puerta del Norte,bc7,622a7233fbcb750090b90e0b,2022-09-28 11:15:00-05:00
7,BC 291 - Las Palmas,bc-291-las-palmas,629a2452dc4c660b40c0135f,2022-10-04 13:24:00-05:00
8,BC 302 - Quebrada Seca,bc61,622b4c877ed11a0050150da9,2022-10-04 13:27:00-05:00
9,BC 306 - Barrancabermeja,bc16,622a7257fbcb750090b90f10,2022-10-04 13:27:00-05:00
